In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [ ]:
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 1

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [ ]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [ ]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [ ]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH)

Attack dataset was already established previosly, loading dataset from "pickle/cifar10/resnet34/attack_train.p".


In [ ]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 3.9567e+00,  2.2521e+00,  1.1713e+00]],  [[ 8.7314e+00,  2.3901e+00, -1.1134e-01]],  [[ 9.1530e+00, -5.8557e-01, -9.3829e-01]],  [[ 4.0542e+00,  4.8081e-01, -2.4173e-02]],  [[ 1.1194e+01,  1.3065e+00, -2.6194e-02]],  [[ 1.1535e+01, -2.0299e-01, -1.5129e+00]],  [[ 2.5223e+00,  1.8923e+00,  9.7421e-01]],  [[ 3.9389e+00,  2.3877e+00,  1.7223e+00]],  [[ 9.7619e+00,  9.6083e-02, -1.0826e-01]],  [[ 8.4934e+00,  9.1026e-01,  1.8671e-01]],  [[ 7.3696e+00,  7.4837e-01,  4.6259e-02]],  [[ 9.0239e+00,  1.9236e+00,  1.3797e-01]],  [[ 1.7865e+00,  1.2105e+00,  1.2056e+00]],  [[ 7.3080e+00,  2.8119e+00,  1.3903e+00]],  [[ 6.9371e+00,  3.4246e+00,  2.5515e+00]],  [[ 1.6093e+01,  7.8114e-01, -2.6410e+00]],  [[ 3.5276e+00,  1.3736e+00, -1.4739e-01]],  [[ 4.4020e+00,  4.1027e-01,  1.4436e-02]],  [[ 1.1640e+01, -1.4672e-01, -3.2149e-01]],  [[ 5.5210e+00,  1.9858e+00, -4.8117e-01]],  [[ 2.5924e+00,  1.3980e+00,  2.6303e-01]],  [[ 3.6911e+00,  2.8068e-01, -1.9508e-01]],  [[ 4.2035e+00,  3.8556e

### Attack Model

Simple Model

In [ ]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [ ]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [ ]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [ ]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3373, 0.3765, 0.3843,  ..., 0.2078, 0.2118, 0.2235], [0.4392, 0.5412, 0.5059,  ..., 0.2510, 0.2549, 0.2235], [0.4275, 0.4510, 0.4471,  ..., 0.2627, 0.2549, 0.2314], ..., [0.5373, 0.5647, 0.5529,  ..., 0.3490, 0.2745, 0.1765], [0.4157, 0.4353, 0.4549,  ..., 0.1608, 0.0941, 0.0745], [0.3882, 0.4000, 0.3961,  ..., 0.0627, 0.0627, 0.0627]],[[0.3765, 0.4314, 0.4314,  ..., 0.3412, 0.3333, 0.3490], [0.5529, 0.6784, 0.6275,  ..., 0.4039, 0.4157, 0.3804], [0.5059, 0.5569, 0.5765,  ..., 0.4235, 0.4118, 0.3843], ..., [0.6627, 0.7059, 0.7059,  ..., 0.3843, 0.2902, 0.1843], [0.4824, 0.5137, 0.5373,  ..., 0.1843, 0.1098, 0.0824], [0.4431, 0.4627, 0.4627,  ..., 0.0745, 0.0745, 0.0745]],[[0.1882, 0.2157, 0.2549,  ..., 0.4980, 0.4941, 0.5176], [0.1882, 0.2941, 0.2745,  ..., 0.5647, 0.5882, 0.5490], [0.3804, 0.2941, 0.2353,  ..., 0.5843, 0.5804, 0.5569], ..., [0.7765, 0.8314,

Attack Evaluation DL Sample

In [ ]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 8.2969e+00,  4.9705e-01,  3.0791e-01]],  [[ 1.0034e+01, -5.1437e-01, -6.1488e-01]],  [[ 9.3802e+00,  4.1555e-01, -4.3127e-01]],  [[ 1.0471e+01, -8.2182e-02, -4.2400e-01]],  [[ 9.2651e+00,  4.1612e-02,  2.7437e-02]],  [[ 9.0344e+00,  5.6719e-01, -8.2624e-01]],  [[ 7.8524e+00,  4.1818e-01, -1.2401e-01]],  [[ 9.0963e+00,  4.6028e-01, -6.6548e-02]],  [[ 9.3732e+00,  8.5999e-02, -3.7518e-01]],  [[ 8.2181e+00,  6.9449e-01,  4.7225e-01]],  [[ 9.8387e+00,  5.3789e-01,  1.6403e-01]],  [[ 1.1495e+01, -4.4880e-01, -5.2808e-01]],  [[ 8.1442e+00,  6.3936e-02, -5.5746e-01]],  [[ 8.8131e+00, -5.3464e-02, -5.1176e-01]],  [[ 8.9089e+00, -2.0440e-01, -2.7369e-01]],  [[ 8.3238e+00,  3.2423e-01, -2.2219e-01]],  [[ 9.1166e+00,  6.2863e-01, -3.0783e-01]],  [[ 1.0020e+01,  3.4747e-01, -5.6266e-01]],  [[ 8.7493e+00,  2.3795e-01,  1.7270e-01]],  [[ 9.6765e+00, -3.6445e-02, -4.0399e-01]],  [[ 9.6246e+00,  8.5195e-01, -1.9971e-01]],  [[ 9.2846e+00,  1.2553e-01, -2.5514e-01]],  [[ 1.0460e+01,  1.0478e

### Train Attack model

In [ ]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 30000
Batchsize: 64
Inputs:tensor([[[ 1.5482e+01, -1.1628e+00, -1.8282e+00]],  [[ 1.1281e+01,  3.8217e+00, -1.9052e+00]],  [[ 1.2290e+01,  9.8166e-01, -1.2734e+00]],  [[ 6.6359e+00,  4.1631e-01,  2.3707e-01]],  [[ 3.7553e+00,  1.1720e+00,  5.5990e-01]],  [[ 1.1729e+01, -4.6245e-01, -7.9813e-01]],  [[ 7.5011e+00,  1.5680e+00,  8.9420e-01]],  [[ 6.1661e+00,  1.0230e+00, -3.3113e-01]],  [[ 8.7063e+00,  6.9468e+00, -2.2595e+00]],  [[ 3.4986e+00,  1.7438e+00,  5.7986e-02]],  [[ 2.4196e+00,  1.7815e+00,  1.6315e+00]],  [[ 4.3901e+00,  1.9657e+00,  4.9074e-01]],  [[ 9.1401e+00,  6.6314e+00, -2.2974e+00]],  [[ 6.3961e+00,  6.7163e-02, -1.0543e-01]],  [[ 5.2773e+00,  1.6130e+00, -4.4109e-01]],  [[ 1.3451e+01, -1.9795e+00, -2.1000e+00]],  [[ 9.4836e+00,  4.7044e+00, -6.5308e-01]],  [[ 4.4989e+00,  2.2623e+00,  1.7637e+00]],  [[ 4.1425e+00,  3.5118e+00, -9.0276e-02]],  [[ 9.5909e+00,  7.2572e+00,

### Evaluation

In [ ]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 28.56
